In [ ]:
# Import Initial Python Librarys.
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
%matplotlib inline


In [ ]:
# Load .env enviroment variables
load_dotenv()

## Part 1 - Personal Finance Planner

Collect Crypto Prices Using the requests Library

In [ ]:
# Set current amount of crypto assets
my_btc = 1.2
my_eth = 5.3

In [ ]:
# Crypto API URLs according to Requerments
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [ ]:
#response_data = requests.get(btc_url).json ( Using requests and Json Python Module)
response_data_btc = requests.get(btc_url)

response_data_btc

response_content_btc = response_data_btc.content

response_content_btc

data_btc = response_data_btc.json()

import json

print(json.dumps(data_btc, indent=4))

response_data_eth = requests.get(eth_url)

response_content_eth = response_data_eth.content

data_eth = response_data_eth.json()

import json
print(json.dumps(data_eth, indent=4))

In [ ]:
# Fetch current price

# Compute current value of my crpto

btc_value = data_btc["data"]["1"]["quotes"]["USD"]["price"]

my_btc_value = my_btc * btc_value

eth_value = data_eth["data"]["1027"]["quotes"]["USD"]["price"]

my_eth_value = my_eth * eth_value

# Print current crypto wallet balance


print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")

print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")


Collect Investments Data Using Alpaca: SPY (stocks) and AGG (bonds)

In [ ]:
# Current amount of shares.

my_agg = 200
my_spy = 50

In [ ]:
# Set Alpaca API key and secret

alpaca_api_key = os.getenv("ALPACA_API_KEY")

alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")


In [ ]:
# Create the Alpaca API object

api = tradeapi.REST(alpaca_api_key,alpaca_secret_key, api_version = "v2")

In [ ]:
# Format current date as ISO format
today = pd.Timestamp("2020-10-16", tz="America/New_York").isoformat()

# Set the tickers

tickers = ["AGG", "SPY"]

# Set timeframe to '1D' for Alpaca API

timeframe = "1D"

# Get current closing prices for SPY and AGG

df_portfolio = api.get_barset( tickers, timeframe, start=today, end=today,).df

# Drop the time component of the date

df_portfolio.index = df_portfolio.index.date

# Preview DataFrame

df_portfolio.head()

In [ ]:
# Pick AGG and SPY close prices

agg_close_price = float(df_portfolio["AGG"]["close"])

spy_close_price = float(df_portfolio["SPY"]["close"])

# Print AGG and SPY close prices

print(f"Current AGG closing price: ${agg_close_price}")

print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
# Compute the current value of shares

my_agg_value = my_agg * agg_close_price 

my_spy_value = my_spy * spy_close_price 

# Print current value of share

print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")

print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

Savings Health Analysis

In [ ]:
# Set monthly household income

monthly_income = 12000

total_crypto = my_btc_value + my_eth_value

total_shares_value = my_agg_value + my_spy_value

data = {'amount':[total_crypto, total_shares_value]} 

# Creates pandas DataFrame.

df_savings = pd.DataFrame(data, index =['crypto', 'shares']) 

# print the data 

df_savings

In [ ]:
# Plot savings pie chart

df_savings.plot.pie(y="amount", title="Composition of Personal Savings")

In [ ]:
# Set ideal emergency fund

emergency_fund = monthly_income * 3

# Calculate total amount of savings

total_savings = total_crypto + total_shares_value

# Validate saving health

if emergency_fund < total_savings:
        print("Congratuations! You have enough money in your emergency fund.")
elif emergency_fund == total_savings:
        print("Congratulations! You have reach this financial goal")
else:
        print(f"You are {emergency_fund - total_savings} dollars away from reaching the goal")

## Part 2 - Retirement Planning

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen

start_date = pd.Timestamp('2015-10-16', tz='America/New_York').isoformat()

end_date = pd.Timestamp('2020-10-16', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG

tickers = ["AGG","SPY"]

df_stock_data = api.get_barset(tickers, timeframe, start=start_date, end=end_date).df

# Drop the time component of the date

df_stock_data.index = df_stock_data.index.date

# Display sample data

df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns

MC_even_dist = MCSimulation(portfolio_data = df_stock_data, weights = [.40,.60], num_simulation = 500, num_trading_days = 252*30)

In [ ]:
# Printing the simulation input data

MC_even_dist.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns

MC_even_dist.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes

line_plot = MC_even_dist.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals

dist_plot = MC_even_dist.plot_distribution()

Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results

even_tbl = MC_even_dist.summarize_cumulative_return()

# Print summary statistics

print(even_tbl)

Calculate the expected portfolio return at the 95% lower and upper confidence intervals based on a $20,000 initial investment

In [ ]:
# Set initial investment

initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000

ci_lower = round(even_tbl[8]*20000,2)

ci_upper = round(even_tbl[9]*20000,2)

# Print results

print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

Calculate the expected portfolio return at the 95% lower and upper confidence intervals based on a 50% increase in the initial investment.

In [ ]:
# Set initial investment

initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000

ci_lower = round(even_tbl[8]*30000,2)

ci_upper = round(even_tbl[9]*30000,2)

# Print results

print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Optional Challenge - Early Retirement

Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns

MC_even_dist_five = MCSimulation(portfolio_data = df_stock_data, weights = [.40,.60], num_simulation = 500, num_trading_days = 252*5)

MC_even_dist_five.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns

MC_even_dist_five.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes

line_plot_five = MC_even_dist_five.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals

dist_plot_five = MC_even_dist_five.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results

even_tbl_five = MC_even_dist_five.summarize_cumulative_return()

# Print summary statistics

print(even_tbl_five)

In [ ]:
# Set initial investment

initial_investment_five = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000

ci_lower_five = round(even_tbl_five[8]*60000,2)

ci_upper_five = round(even_tbl_five[9]*60000,2)

# Print results

print(f"There is a 95% chance that an initial investment of ${initial_investment_five} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns

MC_even_dist_ten = MCSimulation(portfolio_data = df_stock_data, weights = [.40,.60], num_simulation = 500, num_trading_days = 252*10)

MC_even_dist_ten.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns

MC_even_dist_ten.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes

line_plot_ten = MC_even_dist_ten.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals

dist_plot_ten = MC_even_dist_ten.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results

even_tbl_ten = MC_even_dist_ten.summarize_cumulative_return()

# Print summary statistics

print(even_tbl_ten)

In [ ]:
# Set initial investment

initial_investment_ten = 60000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000

ci_lower_ten = round(even_tbl_ten[8]*60000,2)

ci_upper_ten = round(even_tbl_ten[9]*60000,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment_ten} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")